In [ ]:
# -------------------
# Coverage distribution
# -------------------

import cv2
import numpy as np
import os
import torch
from datetime import datetime
import pandas as pd

# SAHI
from sahi import AutoDetectionModel
from sahi.predict import get_sliced_prediction

# Ultralytics SAM
from ultralytics import SAM

import matplotlib.pyplot as plt

# -------------------
save_dir = "/mnt/c/Users/user/Desktop/cnn/segment-anything-2/runs"
os.makedirs(save_dir, exist_ok=True)

# Time stamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
print(f"Timestamp for this run: {timestamp}")

# YOLO model path
yolo_model_path = "/mnt/c/Users/user/Desktop/cnn/segment-anything-2/runs/detect/train34/weights/best.pt"
image_path = "/mnt/c/Users/user/Desktop/keti_cpsem/CAM84/5000.jpg"

# DetectionModel instance for SAHI
detection_model = AutoDetectionModel.from_pretrained(
    model_type='ultralytics',
    model_path=yolo_model_path,
    confidence_threshold=0.3, # ADJUST AS A PARAMETER
    device="cuda:0" if torch.cuda.is_available() else "cpu",
)

# SAM model load
sam_model = SAM("sam3.pt") # YOU CAN USE SAM2 model or SAM3 model (from META AI)
sam_model.to(detection_model.device)

# class color
CAM_COLOR  = [0, 255, 0]  # Green
VOID_COLOR = [0, 0, 255] # Red

# CAM: #91A7A4 (RGB: 145, 167, 164) -> BGR: [164, 167, 145]
CAM_COLOR_BGR  = [164, 167, 145]   
# VOID: #EF4081 (RGB: 239, 64, 129) -> BGR: [129, 64, 239]
VOID_COLOR_BGR = [129, 64, 239]    
# SE:   #EEC843 (RGB: 238, 200, 67) -> BGR: [67, 200, 238]
SE_COLOR_BGR   = [67, 200, 238]  


# YOLO predict
print("\nRunning SAHI Sliced Inference for YOLO detections...")
sahi_yolo_results = get_sliced_prediction(
    image=cv2.imread(image_path),
    detection_model=detection_model,
    slice_height=600, # ADJUST AS A PARAMETER
    slice_width=600, # ADJUST AS A PARAMETER
    overlap_height_ratio=0.4, # ADJUST AS A PARAMETER
    overlap_width_ratio=0.4 # ADJUST AS A PARAMETER
)
object_prediction_list = sahi_yolo_results.object_prediction_list
print(f"SAHI Sliced Inference complete. Found {len(object_prediction_list)} objects.")

# SAM segmentation
img = cv2.imread(image_path)
cam_mask  = np.zeros_like(img[:, :, 0], dtype=bool)
void_mask = np.zeros_like(img[:, :, 0], dtype=bool)

if len(object_prediction_list) > 0:
    batch_size = 500
    print(f"\nRunning SAM with tiled YOLO detections in batches of {batch_size}...")
    for i in range(0, len(object_prediction_list), batch_size):
        batch_predictions = object_prediction_list[i:i+batch_size]
        batch_boxes = []
        batch_class_ids = []
        for pred in batch_predictions:
            bbox = pred.bbox
            batch_boxes.append([bbox.minx, bbox.miny, bbox.maxx, bbox.maxy])
            batch_class_ids.append(pred.category.id)
        batch_boxes_tensor = torch.tensor(batch_boxes, device=detection_model.device)
        sam_results_batch = sam_model(img, bboxes=batch_boxes_tensor, verbose=False, conf=0.01, iou=0.2, retina_masks=True, mask_ratio=4) # ADJUST AS A PARAMETER

        if sam_results_batch and sam_results_batch[0].masks is not None:
            for mask_tensor, class_id in zip(sam_results_batch[0].masks.data, batch_class_ids):
                mask_np = mask_tensor.cpu().numpy()
                if class_id == 0:
                    cam_mask  = np.logical_or(cam_mask, mask_np)
                elif class_id == 1:
                    void_mask = np.logical_or(void_mask, mask_np)
        torch.cuda.empty_cache()
        print(f"Processed batch {i//batch_size + 1}/{(len(object_prediction_list) + batch_size - 1)//batch_size}")
    print("SAM segmentation complete.")

    # -------------------
    print("\nVisualizing results and calculating coverage...")
    colored_mask = np.zeros_like(img)
    colored_mask[cam_mask]  = CAM_COLOR
    colored_mask[void_mask] = VOID_COLOR
    alpha = 0.5
    result_img = cv2.addWeighted(img, 1, colored_mask, alpha, 0)
    seg_save_path = os.path.join(save_dir, f"segmentation_result_{timestamp}.png")
    cv2.imwrite(seg_save_path, result_img)
    print(f"Segmentation result saved: {seg_save_path}")
    cam_mask_u8  = (cam_mask * 255).astype(np.uint8)
    void_mask_u8 = (void_mask * 255).astype(np.uint8)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(cam_mask_u8, connectivity=8)
    kernel = np.ones((3, 3), np.uint8)
    void_mask_dilated = cv2.dilate(void_mask_u8, kernel)
    valid_coverages = []
    for label_id in range(1, num_labels):
        object_mask = (labels == label_id)
        x, y, w, h, area = stats[label_id]
        img_h, img_w = cam_mask.shape[:2]
        if (x == 0) or (x + w >= img_w) or (y == 0) or (y + h >= img_h):
            continue
        eroded_object = cv2.erode(object_mask.astype(np.uint8), kernel)
        boundary = object_mask & ~eroded_object.astype(bool)
        perimeter = np.count_nonzero(boundary)
        if perimeter == 0:
            continue
        boundary_u8 = (boundary * 255).astype(np.uint8)
        boundary_touching_void = cv2.bitwise_and(boundary_u8, void_mask_dilated)
        touching_void_pixels = np.count_nonzero(boundary_touching_void)
        coverage_i = (1 - touching_void_pixels / perimeter) * 100.0
        valid_coverages.append(coverage_i)
    coverage_mean = np.mean(valid_coverages) if valid_coverages else 0.0
    print("\nCoverage results:")
    for idx, cov_val in enumerate(valid_coverages):
        print(f" - CAM object {idx+1} coverage: {cov_val:.2f}%")
    print(f" => Overall coverage for image: {coverage_mean:.2f}%\n")

    # -------------------
    if valid_coverages:
        coverage_data = {
            'CAM Object Number': range(1, len(valid_coverages) + 1),
            'Coverage (%)': valid_coverages
        }
        df = pd.DataFrame(coverage_data)

        df['Coverage (%)'] = df['Coverage (%)'].map('{:.2f}'.format)

        csv_save_path = os.path.join(save_dir, f"coverage_results_{timestamp}.csv")
        df.to_csv(csv_save_path, index=False)
        print(f"Coverage results saved to CSV: {csv_save_path}")
        # ----------------------------------------------------
        print("\nPlotting coverage distribution histogram...")
        plt.figure(figsize=(10, 6)) 
        bins = np.arange(0, 105, 5)
        plt.hist(valid_coverages, bins=bins, edgecolor='black', color='skyblue')
        plt.xlabel("Coverage (%)")
        plt.ylabel("Number of CAM Objects")
        plt.xticks(bins)
        plt.title("Distribution of Individual CAM Object Coverage")
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.tight_layout()
        hist_save_path = os.path.join(save_dir, f"coverage_histogram_{timestamp}.png")
        plt.savefig(hist_save_path)
        plt.show()
        print(f"Histogram saved: {hist_save_path}")
    else:
        print("No valid CAM objects found to save or plot.")
else:
    print("No objects detected by YOLO with SAHI Sliced Inference.")

In [ ]:
# -------------------
# Phase fraction
# -------------------

import numpy as np
import matplotlib.pyplot as plt
import os

if 'cam_mask' in locals() and 'void_mask' in locals():
    print("\nCalculating and plotting phase fractions...")
    
    # [FIX] (BGR -> RGB)
    if 'CAM_COLOR_BGR' in locals() and 'VOID_COLOR_BGR' in locals():
        CAM_COLOR = CAM_COLOR_BGR[::-1]   
        VOID_COLOR = VOID_COLOR_BGR[::-1] 
    else:
        print("Warning: Color variables not found. Using default colors.")
        CAM_COLOR = [64, 64, 64]    # Dark Gray
        VOID_COLOR = [255, 0, 0]    # Red
    SE_COLOR = [128, 128, 128]

    original_void_count = np.sum(void_mask)
    void_mask = np.logical_and(void_mask, np.logical_not(cam_mask))
    
    modified_void_count = np.sum(void_mask)
    overlap_removed = original_void_count - modified_void_count
    
    if overlap_removed > 0:
        print(f"-> Overlap handling applied: {overlap_removed} pixels removed from Void mask (assigned to CAM).")

    total_area = img.shape[0] * img.shape[1]
    
    cam_area = np.sum(cam_mask)
    void_area = np.sum(void_mask) 

    cam_fraction = round((cam_area / total_area) * 100, 2)
    void_fraction = round((void_area / total_area) * 100, 2)
    se_fraction = round(100.00 - cam_fraction - void_fraction, 2)

    print(f"CAM fraction: {cam_fraction:.2f}%")
    print(f"SE fraction: {se_fraction:.2f}% (Calculated as remainder)")
    print(f"Void fraction: {void_fraction:.2f}%")
    
    total_sum = cam_fraction + se_fraction + void_fraction
    print(f"Sum fractions: {total_sum:.2f}% (Forced to 100.00%)")

    phases = ["CAM", "SE", "Void"]
    fractions = [cam_fraction, se_fraction, void_fraction]

    plt.figure(figsize=(8, 6))
    
    bars = plt.bar(phases, fractions, 
                   color=[np.array(CAM_COLOR)/255.0, np.array(SE_COLOR)/255.0, np.array(VOID_COLOR)/255.0],
                   edgecolor='black')
                   
    plt.xlabel("Phase")
    plt.ylabel("Phase Fraction (%)")
    plt.title("Phase Fraction Distribution")
    plt.ylim(0, max(fractions) + 15)
    plt.xticks(phases)

    for bar in bars:
        height_bar = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2.0, height_bar + 1, f"{height_bar:.2f}%", 
                 ha='center', va='bottom', fontsize=10)

    plt.tight_layout()

    if 'save_dir' in locals() and 'timestamp' in locals():
        phase_hist_path = os.path.join(save_dir, f"phase_fraction_histogram_{timestamp}.png")
        plt.savefig(phase_hist_path)
        print(f"Phase fraction histogram saved: {phase_hist_path}")
    
    plt.show()

else:
    print("Main analysis cell (Cell 1) must be run first to generate masks.")

In [ ]:
# -------------------
# Line Intercept Analysis
# -------------------

if 'cam_mask' in locals() and 'void_mask' in locals():
    print("\nStarting Line Intercept Analysis...")
    
    h, w, _ = img.shape
    print(f"Analyzing image with dimensions (Width x Height): {w} x {h} pixels")
    
    from collections import defaultdict
    import random
    import csv

    def generate_systematic_lines(img_shape, line_spacing=1, orientation='both'):
        h, w = img_shape[:2]
        lines = []
        if orientation in ['horizontal', 'both']:
            for y in range(line_spacing, h, line_spacing):
                lines.append([(x, y) for x in range(w)])
        if orientation in ['vertical', 'both']:
            for x in range(line_spacing, w, line_spacing):
                lines.append([(x, y) for y in range(h)])
        return lines

    def classify_pixel(x, y, cam_mask, void_mask):
        if cam_mask[y, x]: return 1
        elif void_mask[y, x]: return 2
        else: return 0

    def measure_intercepts(lines, cam_mask, void_mask):
        intercept_lengths = {0: [], 1: [], 2: []}
        for line in lines:
            current_phase = None
            current_length = 0
            for x, y in line:
                if y >= cam_mask.shape[0] or x >= cam_mask.shape[1]: continue
                phase = classify_pixel(x, y, cam_mask, void_mask)
                if current_phase is None:
                    current_phase, current_length = phase, 1
                elif phase == current_phase:
                    current_length += 1
                else:
                    if current_length > 0:
                        intercept_lengths[current_phase].append(current_length)
                    current_phase, current_length = phase, 1
            if current_phase is not None and current_length > 0:
                intercept_lengths[current_phase].append(current_length)
        return intercept_lengths

    def calculate_mean_intercept_length(intercept_lengths):
        mean_lengths = {}
        for phase, lengths in intercept_lengths.items():
            mean_lengths[phase] = np.mean(lengths) if lengths else 0.0
        return mean_lengths

    def visualize_test_lines(image, lines, line_color=(255, 255, 0), thickness=1):
        result_img = image.copy()
        for line in lines:
            if len(line) > 1:
                cv2.polylines(result_img, [np.array(line, dtype=np.int32)], isClosed=False, color=line_color, thickness=thickness)
        return result_img
    
    def create_phase_map(cam_mask, void_mask, lines=None):
        h, w = cam_mask.shape[:2]
        phase_map = np.zeros((h, w, 3), dtype=np.uint8)
        
        phase_map[:] = [128, 128, 128] 
        phase_map[void_mask] = [0, 0, 255] 
        phase_map[cam_mask] = [0, 255, 0] 
        
        if lines:
            phase_map = visualize_test_lines(phase_map, lines, line_color=(255, 255, 0))
        return phase_map

    phase_names = {0: "SE Background", 1: "CAM", 2: "VOID"}
    phase_colors_plt = {0: 'gray', 1: 'green', 2: 'red'}

    line_spacing = 5
    lines = generate_systematic_lines(img.shape, line_spacing=line_spacing, orientation='both')
    print(f"Generated {len(lines)} test lines for intercept analysis.")

    intercept_lengths = measure_intercepts(lines, cam_mask, void_mask)
    total_intercepts = sum(len(lengths) for lengths in intercept_lengths.values())
    print(f"Measured a total of {total_intercepts} intercepts.")

    mean_lengths = calculate_mean_intercept_length(intercept_lengths)
    print("\n===== Line Intercept Analysis Results =====")
    print("Phase\t\tMean Length\tCount")
    print("----------------------------------------------")
    for phase, mean_len in mean_lengths.items():
        name = phase_names[phase]
        tab = "\t" if len(name) >= 12 else "\t\t"
        print(f"{name}{tab}{mean_len:.2f}\t\t{len(intercept_lengths[phase])}")
    print("==============================================")
    
    print("\nGenerating and saving phase map...")
    phase_map_img = create_phase_map(cam_mask, void_mask, lines=lines)
    phase_map_path = os.path.join(save_dir, f"phase_map_{timestamp}.png")
    cv2.imwrite(phase_map_path, phase_map_img)
    print(f"Phase map saved: {phase_map_path}")

    print("\nGenerating and saving intercept length histograms...")
    fig, axs = plt.subplots(len(phase_names), 1, figsize=(10, 9), tight_layout=True)
    all_lengths = [l for lengths in intercept_lengths.values() for l in lengths]
    global_max = max(all_lengths) * 1.05 if all_lengths else 100

    for i, phase in enumerate(sorted(phase_names.keys())):
        lengths = intercept_lengths[phase]
        ax = axs[i] if len(phase_names) > 1 else axs
        if lengths:
            ax.hist(lengths, bins=50, color=phase_colors_plt[phase], alpha=0.7)
            ax.set_title(f"{phase_names[phase]} Intercept Length (Mean: {mean_lengths[phase]:.2f} px)")
        else:
            ax.set_title(f"{phase_names[phase]} (No data)")
        ax.set_ylabel("Frequency")
        ax.set_xlim(0, global_max)
        ax.grid(True, linestyle='--', alpha=0.7)
    
    axs[-1].set_xlabel("Intercept Length (pixels)")
    
    hist_path = os.path.join(save_dir, f"intercept_histograms_{timestamp}.png")
    fig.savefig(hist_path)
    plt.show()
    print(f"Intercept length histograms saved: {hist_path}")

    print("\nExporting intercept length data to CSV...")
    csv_path = os.path.join(save_dir, f"intercept_lengths_{timestamp}.csv")
    with open(csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Phase', 'Intercept Length (pixels)'])
        for phase, lengths in intercept_lengths.items():
            for length in lengths:
                writer.writerow([phase_names[phase], length])
    print(f"Intercept length data exported to CSV: {csv_path}")

    print("\nLine Intercept Analysis complete!")

else:
    print("Main analysis cell (Cell 1) must be run first to generate masks.")

In [ ]:
def snap_added_counts(mask, axis, band):
    m = snap_band_to_edge(mask, axis, band)
    if axis == 0:
        added_top = int((m[0, :] & ~mask[0, :]).sum())
        added_bottom = int((m[-1, :] & ~mask[-1, :]).sum())
        return added_top, added_bottom
    else:
        added_left  = int((m[:, 0] & ~mask[:, 0]).sum())
        added_right = int((m[:, -1] & ~mask[:, -1]).sum())
        return added_left, added_right

print("Y added (band=1):", snap_added_counts(binary_mask, axis=0, band=1))
print("Y added (band=3):", snap_added_counts(binary_mask, axis=0, band=3))
print("X added (band=1):", snap_added_counts(binary_mask, axis=1, band=1))
print("X added (band=3):", snap_added_counts(binary_mask, axis=1, band=3))

In [ ]:
# -------------------
# w/ void Tortuosity Analysis
# -------------------

if 'cam_mask' in locals() and 'void_mask' in locals():
    print("\n===== Starting Tortuosity Analysis (SE Only) =====")

    import os, random
    import numpy as np
    import cv2
    import matplotlib.pyplot as plt
    import porespy as ps
    from skimage import measure, morphology

    try:
        import networkx as nx
        has_networkx = True
    except ImportError:
        print("WARNING: NetworkX module not found. Skipping path analysis.")
        has_networkx = False

    print("Creating a binary mask where only the SE phase is conductive...")
    se_mask = np.logical_not(np.logical_or(cam_mask, void_mask))
    
    print("Dilating the SE mask by pixels to connect nearby regions...")
    kernel = np.ones((3, 3), np.uint8)
    se_mask_u8 = (se_mask * 255).astype(np.uint8)
    dilated_se_mask_u8 = cv2.dilate(se_mask_u8, kernel, iterations=0)
    dilated_se_mask = dilated_se_mask_u8.astype(bool)
    binary_mask = dilated_se_mask

    def snap_band_to_edge(mask: np.ndarray, axis: int, band: int = 1):
        H, W = mask.shape
        m = mask.copy()
        band = max(1, int(band))

        if axis == 0:
            top_band = mask[:band, :]
            cols_top = np.where(top_band.any(axis=0))[0]
            if cols_top.size > 0:
                m[0, cols_top] = True

            bot_band = mask[-band:, :]
            cols_bot = np.where(bot_band.any(axis=0))[0]
            if cols_bot.size > 0:
                m[-1, cols_bot] = True
        else:
            left_band = mask[:, :band]
            rows_left = np.where(left_band.any(axis=1))[0]
            if rows_left.size > 0:
                m[rows_left, 0] = True

            right_band = mask[:, -band:]
            rows_right = np.where(right_band.any(axis=1))[0]
            if rows_right.size > 0:
                m[rows_right, -1] = True
        return m

    print("\nAnalyzing directional tortuosity...")
    
    results_y, results_x = None, None
    tau_y, d_eff_y = float('nan'), float('nan')
    tau_x, d_eff_x = float('nan'), float('nan')

    # Y-direction (axis=0)
    try:
        print("  - Calculating for Y-direction (top-to-bottom)...")
        mask_y = snap_band_to_edge(binary_mask, axis=0, band=0)
        results_y = ps.simulations.tortuosity_fd(mask_y, axis=0)
        tau_y = results_y.tortuosity
        d_eff_y = 1/results_y.formation_factor if results_y.formation_factor > 0 else 0
        top = int(mask_y[0, :].sum()); bot = int(mask_y[-1, :].sum())
        print(f"     [Y-edge true] top={top}, bottom={bot}")
        print(f"  -> Y-direction SUCCESS: τ_y = {tau_y:.4f}, D_eff_y = {d_eff_y:.4f}")
    except Exception as e:
        print(f"  -> Y-direction FAILED: {type(e).__name__}: {e}")

    # X-direction (axis=1)
    try:
        print("  - Calculating for X-direction (left-to-right)...")
        mask_x = snap_band_to_edge(binary_mask, axis=1, band=0)
        results_x = ps.simulations.tortuosity_fd(mask_x, axis=1)
        tau_x = results_x.tortuosity
        d_eff_x = 1/results_x.formation_factor if results_x.formation_factor > 0 else 0
        lef = int(mask_x[:, 0].sum()); rig = int(mask_x[:, -1].sum())
        print(f"     [X-edge true] left={lef}, right={rig}")
        print(f"  -> X-direction SUCCESS: τ_x = {tau_x:.4f}, D_eff_x = {d_eff_x:.4f}")
    except Exception as e:
        print(f"  -> X-direction FAILED: {type(e).__name__}: {e}")

    if results_y is not None or results_x is not None:
        results = results_y if results_y is not None else results_x
        concentration = results.concentration
        effective_porosity = results.effective_porosity
        viz_mask = (mask_y if results_y is not None else mask_x) if ('mask_y' in locals() or 'mask_x' in locals()) else binary_mask
        
        print("\nVisualizing concentration gradient and skeleton...")
        grad_y, grad_x = np.gradient(concentration)
        gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
        masked_gradient = np.ma.masked_where(~viz_mask, gradient_magnitude)
        plt.figure(figsize=(12, 10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.imshow(masked_gradient, cmap='hot', alpha=0.7)
        plt.colorbar(label='Concentration Gradient Magnitude')
        plt.title('Diffusion Path Visualization')
        plt.axis('off'); plt.tight_layout()
        save_path = os.path.join(save_dir, f"gradient_visualization_w_void_{timestamp}.png")
        plt.savefig(save_path, dpi=300); plt.close()
        print(f"Saved: {os.path.basename(save_path)}")

        skeleton = morphology.skeletonize(viz_mask)
        plt.figure(figsize=(12, 10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.imshow(skeleton, cmap='winter', alpha=0.8)
        plt.title('Skeleton of Conductive Region')
        plt.axis('off'); plt.tight_layout()
        save_path = os.path.join(save_dir, f"skeleton_visualization_w_void_{timestamp}.png")
        plt.savefig(save_path, dpi=300); plt.close()
        print(f"Saved: {os.path.basename(save_path)}")

        # NetworkX
        if has_networkx:
            print("\nAnalyzing shortest path using NetworkX...")
            h, w = skeleton.shape
            G = nx.Graph()
            for y in range(h):
                for x in range(w):
                    if skeleton[y, x]:
                        G.add_node(y * w + x, pos=(x, y))
            directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
            for y in range(h):
                for x in range(w):
                    if skeleton[y, x]:
                        node1 = y * w + x
                        for dy, dx in directions:
                            ny, nx_ = y + dy, x + dx
                            if 0 <= ny < h and 0 <= nx_ < w and skeleton[ny, nx_]:
                                node2 = ny * w + nx_
                                weight = 1.414 if abs(dx) == 1 and abs(dy) == 1 else 1.0
                                G.add_edge(node1, node2, weight=weight)
            top_row_nodes = [n for n, d in G.nodes(data=True) if d['pos'][1] < 20]
            bottom_row_nodes = [n for n, d in G.nodes(data=True) if d['pos'][1] > h - 20]
            if top_row_nodes and bottom_row_nodes:
                start_node, end_node = random.choice(top_row_nodes), random.choice(bottom_row_nodes)
                start_x, start_y = G.nodes[start_node]['pos']
                end_x, end_y = G.nodes[end_node]['pos']
                try:
                    shortest_path = nx.shortest_path(G, source=start_node, target=end_node, weight='weight')
                    path_coords = np.array([G.nodes[n]['pos'] for n in shortest_path])
                    path_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    path_coords_xy = path_coords
                    cv2.polylines(path_img, [path_coords_xy], isClosed=False, color=(255, 0, 0), thickness=3)
                    cv2.circle(path_img, (start_x, start_y), 10, (0, 0, 255), -1)
                    cv2.circle(path_img, (end_x, end_y), 10, (0, 255, 0), -1)
                    plt.figure(figsize=(12, 10)); plt.imshow(path_img)
                    plt.title('Shortest Path Through Conductive Region'); plt.axis('off')
                    save_path = os.path.join(save_dir, f"shortest_path_w_void_{timestamp}.png")
                    plt.savefig(save_path, dpi=300); plt.close()
                    print(f"Saved: {os.path.basename(save_path)}")
                    direct_distance = np.sqrt((end_y - start_y)**2 + (end_x - start_x)**2)
                    actual_distance = nx.shortest_path_length(G, source=start_node, target=end_node, weight='weight')
                    local_tortuosity = actual_distance / direct_distance if direct_distance > 0 else float('inf')
                    txt_path = os.path.join(save_dir, f"path_analysis_w_void_{timestamp}.txt")
                    with open(txt_path, 'w') as f:
                        f.write(f"Path Analysis Results (SE)\n")
                        f.write(f"Direct distance: {direct_distance:.2f} pixels\n")
                        f.write(f"Actual path length: {actual_distance:.2f} pixels\n")
                        f.write(f"Local tortuosity: {local_tortuosity:.4f}\n")
                    print(f"Saved: {os.path.basename(txt_path)}")
                except nx.NetworkXNoPath:
                    print("No path found between the selected start and end nodes.")
            else:
                print("Could not find start/end nodes for path analysis.")

        print("\nVisualizing directional diffusivity...")
        fig, ax = plt.subplots(figsize=(8, 6))
        bars = ax.bar(['X-direction', 'Y-direction'], [d_eff_x, d_eff_y], color=['blue', 'orange'])
        ax.set_ylabel('Effective Diffusivity')
        ax.set_title('Directional Effective Diffusivity')
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height, f'{height:.4f}', ha='center', va='bottom')
        save_path = os.path.join(save_dir, f"directional_diffusivity_w_void_{timestamp}.png")
        plt.savefig(save_path, dpi=300); plt.close()
        print(f"Saved: {os.path.basename(save_path)}")
        
        print("\nAnalyzing connected region size distribution...")
        labeled_mask, num_features = measure.label(viz_mask, return_num=True)
        region_sizes = [np.sum(labeled_mask == i) for i in range(1, num_features + 1)]
        largest_percentage = 0
        if region_sizes:
            region_sizes.sort(reverse=True)
            largest_percentage = region_sizes[0] / np.sum(viz_mask) * 100
            plt.figure(figsize=(10, 6))
            max_regions = min(10, len(region_sizes))
            plt.bar(range(1, max_regions + 1), region_sizes[:max_regions])
            plt.xlabel('Region Rank'); plt.ylabel('Size (pixels)')
            plt.title(f'Top {max_regions} Conductive Regions by Size (Dilated SE by 2px)')
            save_path = os.path.join(save_dir, f"region_sizes_w_void_{timestamp}.png")
            plt.savefig(save_path, dpi=300); plt.close()
            print(f"Saved: {os.path.basename(save_path)}")
        else:
            num_features = 0
            print("No connected regions found in the SE mask.")
            
        print("\nGenerating final summary report...")
        plt.figure(figsize=(10, 8))
        summary_text = (
            f"TORTUOSITY ANALYSIS SUMMARY (Image w/ Void)\n"
            f"=================================================================\n\n"
            f"Porosity (φ) of Conductive Phase: {effective_porosity:.4f}\n\n"
            f"Directional Analysis:\n"
            f"  - Y-Direction (top-bottom): τ_y = {tau_y:.4f}, D_eff_y = {d_eff_y:.4f}\n"
            f"  - X-Direction (left-right): τ_x = {tau_x:.4f}, D_eff_x = {d_eff_x:.4f}\n\n"
            f"Connectivity Analysis:\n"
            f"  - Total Connected Regions: {num_features}\n"
            f"  - Largest Region: {largest_percentage:.2f}% of total SE area\n"
        )
        plt.text(0.05, 0.95, summary_text, ha='left', va='top', fontsize=12, fontfamily='monospace',
                 bbox=dict(boxstyle='round,pad=1', facecolor='lightblue', alpha=0.5))
        plt.axis('off'); plt.tight_layout()
        save_path = os.path.join(save_dir, f"summary_report_w_void_{timestamp}.png")
        plt.savefig(save_path, dpi=300); plt.close()
        print(f"Saved: {os.path.basename(save_path)}")

        print("\nAdvanced analysis complete!")

    else:
        print("\n" + "="*50)
        print("ANALYSIS FAILED: The material does not percolate in ANY direction (X or Y).")
        print("Cannot proceed with further analysis.")
        print("="*50 + "\n")
        debug_path = os.path.join(save_dir, f"failed_percolation_mask_w_void_{timestamp}.png")
        cv2.imwrite(debug_path, (binary_mask * 255).astype(np.uint8))
        print(f"The non-percolating mask used for analysis was saved to: {debug_path}")

else:
    print("Main analysis cell (Cell 1) must be run first to generate masks.")

In [ ]:
# -------------------
# w/o void Tortuosity Analysis
# -------------------

if 'cam_mask' in locals() and 'void_mask' in locals():
    print("\n===== Starting Tortuosity Analysis (SE + Void) =====")

    from skimage import measure, morphology
    try:
        import networkx as nx
        has_networkx = True
    except ImportError:
        print("WARNING: NetworkX module not found. Skipping path analysis.")
        has_networkx = False

    print("Creating a binary mask where SE and Void are the conductive path...")
    se_plus_void_mask = np.logical_not(cam_mask)
    
    print("Dilating the SE+Void mask by pixels...")
    kernel = np.ones((3, 3), np.uint8)
    mask_u8 = (se_plus_void_mask * 255).astype(np.uint8)
    dilated_mask_u8 = cv2.dilate(mask_u8, kernel, iterations=0)
    dilated_mask = dilated_mask_u8.astype(bool)
    binary_mask = dilated_mask
    
    print("\nAnalyzing directional tortuosity...")
    
    results_y, results_x = None, None
    tau_y, d_eff_y = float('nan'), float('nan')
    tau_x, d_eff_x = float('nan'), float('nan')

    try:
        print("  - Calculating for Y-direction (top-to-bottom)...")
        results_y = ps.simulations.tortuosity_fd(binary_mask, axis=0)
        tau_y = results_y.tortuosity
        d_eff_y = 1/results_y.formation_factor if results_y.formation_factor > 0 else 0
        print(f"  -> Y-direction SUCCESS: τ_y = {tau_y:.4f}, D_eff_y = {d_eff_y:.4f}")
    except Exception as e:
        print(f"  -> Y-direction FAILED: The material does not percolate in this direction.")

    try:
        print("  - Calculating for X-direction (left-to-right)...")
        results_x = ps.simulations.tortuosity_fd(binary_mask, axis=1)
        tau_x = results_x.tortuosity
        d_eff_x = 1/results_x.formation_factor if results_x.formation_factor > 0 else 0
        print(f"  -> X-direction SUCCESS: τ_x = {tau_x:.4f}, D_eff_x = {d_eff_x:.4f}")
    except Exception as e:
        print(f"  -> X-direction FAILED: The material does not percolate in this direction.")

    if results_y is not None or results_x is not None:
        results = results_y if results_y is not None else results_x
        concentration = results.concentration
        effective_porosity = results.effective_porosity
        
        print("\nVisualizing concentration gradient and skeleton...")
        grad_y, grad_x = np.gradient(concentration)
        gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)
        masked_gradient = np.ma.masked_where(~binary_mask, gradient_magnitude)
        plt.figure(figsize=(12, 10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.imshow(masked_gradient, cmap='hot', alpha=0.7)
        plt.colorbar(label='Concentration Gradient Magnitude')
        plt.title('Diffusion Path Visualization (SE + Void)')
        plt.axis('off'); plt.tight_layout()
        save_path = os.path.join(save_dir, f"gradient_visualization_wo_void_{timestamp}.png")
        plt.savefig(save_path, dpi=300); plt.close()
        print(f"Saved: {os.path.basename(save_path)}")

        skeleton = morphology.skeletonize(binary_mask)
        plt.figure(figsize=(12, 10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.imshow(skeleton, cmap='winter', alpha=0.8)
        plt.title('Skeleton of Conductive Region (SE + Void)')
        plt.axis('off'); plt.tight_layout()
        save_path = os.path.join(save_dir, f"skeleton_visualization_wo_void_{timestamp}.png")
        plt.savefig(save_path, dpi=300); plt.close()
        print(f"Saved: {os.path.basename(save_path)}")
        
        if has_networkx:
            print("\nAnalyzing shortest path using NetworkX...")
            h, w = skeleton.shape
            G = nx.Graph()
        
        print("\nVisualizing directional diffusivity...")
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.bar(['X-direction', 'Y-direction'], [d_eff_x, d_eff_y], color=['blue', 'orange'])
        ax.set_ylabel('Effective Diffusivity')
        ax.set_title('Directional Effective Diffusivity (SE + Void)')
        save_path = os.path.join(save_dir, f"directional_diffusivity_wo_void_{timestamp}.png")
        plt.savefig(save_path, dpi=300); plt.close()
        print(f"Saved: {os.path.basename(save_path)}")
        
        print("\nAnalyzing connected region size distribution...")
        labeled_mask, num_features = measure.label(binary_mask, return_num=True)
        region_sizes = [np.sum(labeled_mask == i) for i in range(1, num_features + 1)]
        largest_percentage = 0
        if region_sizes:
            region_sizes.sort(reverse=True)
            largest_percentage = region_sizes[0] / np.sum(binary_mask) * 100
            plt.figure(figsize=(10, 6))
            plt.bar(range(1, min(11, len(region_sizes)+1)), region_sizes[:10])
            plt.xlabel('Region Rank'); plt.ylabel('Size (pixels)')
            plt.title('Top 10 Conductive Regions by Size (SE + Void)')
            save_path = os.path.join(save_dir, f"region_sizes_wo_void_{timestamp}.png")
            plt.savefig(save_path, dpi=300); plt.close()
            print(f"Saved: {os.path.basename(save_path)}")
        else: num_features = 0
            
        print("\nGenerating final summary report...")
        plt.figure(figsize=(10, 8))
        summary_text = (
            f"TORTUOSITY ANALYSIS SUMMARY (Dilated SE + Void)\n"
            f"================================================\n\n"
            f"Porosity (φ) of Conductive Phase: {effective_porosity:.4f}\n\n"
            f"Directional Analysis:\n"
            f"  - Y-Direction (top-bottom): τ_y = {tau_y:.4f}, D_eff_y = {d_eff_y:.4f}\n"
            f"  - X-Direction (left-right): τ_x = {tau_x:.4f}, D_eff_x = {d_eff_x:.4f}\n\n"
            f"Connectivity Analysis:\n"
            f"  - Total Connected Regions: {num_features}\n"
            f"  - Largest Region: {largest_percentage:.2f}% of total conductive area\n"
        )
        plt.text(0.05, 0.95, summary_text, ha='left', va='top', fontsize=12, fontfamily='monospace',
                 bbox=dict(boxstyle='round,pad=1', facecolor='lightseagreen', alpha=0.5))
        plt.axis('off'); plt.tight_layout()
        save_path = os.path.join(save_dir, f"summary_report_wo_void_{timestamp}.png")
        plt.savefig(save_path, dpi=300); plt.close()
        print(f"Saved: {os.path.basename(save_path)}")

        print("\nanalysis (SE + Void) complete!")

    else:
        print("\n" + "="*50)
        print("ANALYSIS FAILED: The material does not percolate in ANY direction (X or Y)")
        print("Cannot proceed with further analysis.")
        print("="*50 + "\n")
        debug_path = os.path.join(save_dir, f"failed_percolation_mask_wo_void_{timestamp}.png")
        cv2.imwrite(debug_path, (binary_mask * 255).astype(np.uint8))
        print(f"The non-percolating mask used for analysis was saved to: {debug_path}")

else:
    print("Main analysis cell (Cell 1) must be run first to generate masks.")